In [93]:
import numpy as np 
import pandas as pd 
import os

In [94]:
# Paths
BASE_DIR = '../'
LGBM = os.path.join(BASE_DIR, "LGBM/prediccion.csv")
Promedio= os.path.join(BASE_DIR, "Dataset/df_12meses_avg.csv")
LGBM_clase=os.path.join(BASE_DIR, "LGBM/prediccion (0_269).csv") 
LGBM_GPU = os.path.join(BASE_DIR, "LGBM/prediccion_lgbm_gpu.csv")
Regresion_L=os.path.join(BASE_DIR, "LGBM/predicciones_201902_reg_lineal2.csv")

In [95]:
# Datos Tabulares
df1 = pd.read_csv(LGBM)
df2= pd.read_csv(Promedio)
df3= pd.read_csv(LGBM_clase)
df4= pd.read_csv(LGBM_GPU)
df5= pd.read_csv(Regresion_L)

In [96]:
df1.head()

,product_id,tn
0,20001,1623.852832
1,20002,1238.299737
2,20003,707.044906
3,20004,567.090490
4,20005,536.373309


In [97]:
df2.head()

,product_id,tn
0,20001,1454.732720
1,20002,1175.437142
2,20003,784.976407
3,20004,627.215328
4,20005,668.270104


In [98]:
df3.head()

,product_id,tn
0,20001,1582.165992
1,20002,1179.451613
2,20003,633.528487
3,20004,525.096322
4,20005,546.940110


In [99]:
# concatenar los dos DataFrames
df = pd.concat([df4, df2,df3,df5])

# agrupar por product_id y calcular promedio
df_promedio = df.groupby('product_id', as_index=False)['tn'].mean()

print(df_promedio)

     product_id           tn
0         20001  1446.128403
1         20002  1209.838608
2         20003   705.611372
3         20004   576.465132
4         20005   590.809686
..          ...          ...
775       21263     0.527810
776       21265     0.387806
777       21266     0.327772
778       21267     0.402697
779       21276     0.370889

[780 rows x 2 columns]


In [100]:
df_promedio.to_csv('prediccion_ensemble.csv', index=False)

In [101]:
# combinamos todo en un solo DataFrame
df_comb = (
    df1.rename(columns={'tn': 'tn_df1'})
    .merge(df2.rename(columns={'tn': 'tn_df2'}), on='product_id')
    .merge(df3.rename(columns={'tn': 'tn_df3'}), on='product_id')
)

# calculamos el promedio de tn_df1 y tn_df3
df_comb['promedio_1y3'] = df_comb[['tn_df1', 'tn_df3']].mean(axis=1)

# diferencia porcentual en valor absoluto
df_comb['diff_pct_df2_vs_prom'] = (
    100 * (df_comb['tn_df2'] - df_comb['promedio_1y3']) / df_comb['promedio_1y3']
).abs()

# ordenamos por mayor diferencia
df_result = df_comb.reindex(columns=[
    'product_id', 'tn_df1', 'tn_df2', 'tn_df3', 'promedio_1y3', 'diff_pct_df2_vs_prom'
]).sort_values(by='diff_pct_df2_vs_prom', ascending=False)


In [102]:

df_result.head(30)

,product_id,tn_df1,tn_df2,tn_df3,promedio_1y3,diff_pct_df2_vs_prom
618,20886,1.856347,3.522517,0.721215,1.288781,173.321656
633,20919,0.727425,0.947063,0.000000,0.363713,160.387511
192,20236,58.632231,97.616042,35.602598,47.117415,107.176142
595,20835,2.961998,5.786993,2.625595,2.793797,107.137243
770,21248,1.725281,0.025992,0.369185,1.047233,97.518063
772,21256,1.674316,0.025173,0.342966,1.008641,97.504233
560,20772,4.673902,9.344477,4.805692,4.739797,97.149314
775,21263,1.722210,0.029993,0.370596,1.046403,97.133674
774,21262,1.296541,0.027169,0.223423,0.759982,96.425025
773,21259,1.317186,0.029876,0.344378,0.830782,96.403890
